In [ ]:
#========================================================================
# Copyright 2019 Science Technology Facilities Council
# Copyright 2019 University of Manchester
#
# This work is part of the Core Imaging Library developed by Science Technology	
# Facilities Council and University of Manchester
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0.txt
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# 
#=========================================================================

# Reconstruction intro
## FBP, CGLS

**The goal** of this notebook is to get familiar with main Framework concepts through basic filtered-back projection (FBP) and Conjugate Gradient Least Squares (CGLS) reconstructions.

**Learning objectives**

In the end of this session, participants will be able to:
- translate output of a CT instrument into Framework objects
- set-up basic FBP reconstruction
- formulate CT reconstruction as an optimisation problem and solve it iteratively
- visualise final and intermediate reconstruction results

**Prerequisites:**
We expect users to be familiar with Python and object-oriented programming. Here is a couple of useful links which might help to delve into Python smoothly if you are new to Python.
- [A gentle introduction to Python](https://www.programiz.com/python-programming/tutorial)
- [NumPy for MATLAB users cheat sheet](https://mas-dse.github.io/DSE200/cheat_sheets/1_python/6_2_NumPy_for_MATLAB_users.pdf)

In [ ]:
# some imports
# please, ignore this cell for now, 
# this is to fix compatibility issues 
# between different versions of Python
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [ ]:
# import utilities
# to minimise amount of code one needs to write to plot data,
# we implemented some plotting utilities
from utilities import plotter2D

In [ ]:
# set up default device for computation
dev = "gpu" # can be cpu if gou is not available
# set up default colour map for visualisation
cmap = "gray"

## CT data acquisition

In conventional CT systems, an object is placed between a source emitting X-rays and a detector array measuring the  X-ray transmission images of the incident X-rays. Typically, either the object is placed on a rotating sample stage and rotates with respect to the source-detector assembly, or the source-detector gantry rotates with respect to the stationary object.

<img src="figures/parallel.png" width=400 height=400 align="left">

In the Framework, we implemented `AcquisitionGeometry` class to hold acquisition parameters and `ImageGeometry` to hold geometry of a reconstructed volume. Corresponding data arrays are wrapped as `AcquisitionData` and `ImageData` classes, respectively. 

In this notebook we will work with parallel geometry. More complex geometries will be discussed in the following notebooks. Geometrical parameters for parallel geometry are depicted below:

<img src="figures/parallel_geometry.png" width=600 height=600 align="left">

Parallel geometry is 2D geometry, i.e. 2D object is projected onto 1D array. Source-detector rotation with respect to the object (or vice-versa) allows to reconstruct 2D object from 1D measurements. In CT world, data acquired with parallel geometry is quite often referred to as *sinogram*. A sinogram is a stack of 1D projections of a 2D object at different rotation positions. Or, mathemtically speaking, Radon transform of an image.

See [Formation of sinogram](https://www.youtube.com/watch?v=q7Rt_OY_7tU).

As a test object for this notebook we have chosen a very simple 'hotdog' phantom which imitates an object consisting of two materials.

In [ ]:
# imports
from ccpi.framework import TestData
from ccpi.framework import ImageData, ImageGeometry
import os, sys

# load ground truth image
# number of pixels in the ground truth image
N = 512
# initialise loader
loader = TestData(data_dir=os.path.join(sys.prefix, "share", "ccpi"))
# load data
data = loader.load(TestData.SIMPLE_PHANTOM_2D, size=(N, N))
# scale data
data *= 2.5 / N
print(data)

In [ ]:
# plot ground truth image
plotter2D(data,
          "Ground truth image",
          cmap=cmap)

We will load precomputed `AcquisitionData` with corresponding geometry. For simplicity, we will skip simulation step in this notebook. Also we will now use pre-set `AcquisitionGeometry`, we will show later how to manually set-up `AcquisitionGeometry`. 

In [ ]:
# imports
from ccpi.framework import AcquisitionGeometry, AcquisitionData
from ccpi.io import NEXUSDataReader
from ccpi.framework.TestData import data_dir
import numpy as np

# file path
pathname = data_dir
filename = 'sino_ideal.nxs'
path = os.path.join(pathname, filename)

# initialise reader
reader = NEXUSDataReader()
reader.set_up(nexus_file=path)
# load sinogram
ad = reader.load_data()
print(ad)

In [ ]:
# plot AcquistionData
plotter2D(ad,
          "noise free sinogram",
          cmap=cmap)

The sinogram contains 360 rows and 512 columns, consequently projections were acquired over 360 rotation positions using a 1D detector with 512 pixel elements. The corresponding geometry is held in an `AcquisitionGeometry` object:

In [ ]:
# get AcquisitionGeometry
ag = reader.get_geometry()
print(ag)

`AcquisiitonGeometry` class also holds information about arrangement of the actual acquisition data array. We use attribute `dimension_labels` to label axis. The expected dimension labels for `AcquisitionData` and `ImageData` are shown below:

<img src="figures/parallel_data.png" width=300 height=300 align="left">

The default order of dimensions for `AcquisitionData` is `[angle, horizontal]`, meaning that the number of elements along 0 and 1 axes in the acquisition data array is expected to be `n_angles` and `N`, respectively.

In [ ]:
print("Dimension labels:\n{}".format(ag.dimension_labels))

## CT reconstruction
Tomographic reconstruction consists of resolving the three-dimensional photon attenuation map of a scanned object from the collection of projection measurement. There are two major classes of reconstruction algorithms: *analytic* and *iterative*. 

<a id='fbp'></a>
### Analytic reconstruction
The most common analytic reconstruction algorithm is filtered back-projection (FBP). The FBP algorithm is derived from the Fourier Slice theorem which relates line integral measurements to two dimensional Fourier transform of an object’s slice. Although the Fourier Slice theorem provides straightforward solution for tomographic reconstruction, its practical implementation is challenging due to required interpolation from Polar to Cartesian coordinates in the Fourier space. In FBP-type reconstruction methods, projections are ﬁltered independently and then back-projected onto the plane of the tomographic slice. Filtration is used to compensate for nonuniform sampling of the Fourier space (higher frequencies have higher density of sampling points) by linear (Ramp) weighting of the frequency space.

To store reconstruction results, we implemented two classes: `ImageGeometry` and `ImageData`. Similar to `AcquisitionData` and `AcquisitionGeometry`, we first define 2D `ImageGeometry`, i.e. voxel grid on which an image will be reconstructed. 

In [ ]:
# create ImageGeometry 
ig = ImageGeometry(voxel_num_x=ag.pixel_num_h,
                   voxel_size_x=ag.pixel_size_h,
                   voxel_num_y=ag.pixel_num_h,
                   voxel_size_y=ag.pixel_size_h)

FBP algorithm is implemented as a `Processor` which takes as an input `AcquisitionData` along with `AcquisitionGeometry` and `ImageGeometry`, and returns reconstructed `ImageData`.

In [ ]:
# imports
from ccpi.astra.processors import FBP

# reconstruct noise-free data
# configure FBP
fbp = FBP(volume_geometry=ig, 
          sinogram_geometry=ag,
          device=dev)
# pass actual AcquisitionData
fbp.set_input(ad)
# run FBP and get results
recon_fbp_ideal = fbp.process()

In [ ]:
plotter2D(data, 
          "Ground truth",
          cmap=cmap)

plotter2D([recon_fbp_ideal, recon_fbp_ideal-data], 
          ["Reconstruction of noise-free sinogram", "Difference from ground truth"], 
          fix_range=False, 
          stretch_y=False,
          cmap=cmap)

<a id='cgls'></a>
#### Iterative reconstruction
Iterative methods use an initial estimate of volume voxel values which is then iteratively updated to best reproduce acquired radiographic data. Here we discuss formulation of iterative reconstruction for 2D parallel gemetry, extension to other geometries is straightforward. Iterative methods formulate the reconstruction methods as a system of linear equations,

$$Au = b$$

- $u$ is the volume to be reconstructed. $u$ is typically represented as a column vector with $N \cdot N \times 1$ elements, where $N$ is the number of elements in a detector row.
- $b$ is measured data from $M$ measurements (projections), $b$ is a column vector with $N \cdot M \times 1$ elements
- $A$ is the projection operator with $N \cdot M \times N \cdot N$ elements. If $i, i = \{0, 1, \dots N \cdot M - 1 \}$ and $j, j = \{0, 1, \dots, N \cdot N - 1\}$, then $A_{i,j}$ is the length of intersection of the $i$.th ray with the $j$.th voxel.

For any real application, problem size is too large to be solved by direct inversion methods, i.e.

$$u = A^{-1}b$$

Secondly, the projection matrix $A$ is often under-determined (low number of projections or missing angles), i.e. 

$$M \ll N$$

Therefore we formulate reconstruction as an optimization problem and use iterative solvers to solve:

$$\underset{u}{\mathrm{argmin}}\begin{Vmatrix}A u - b\end{Vmatrix}^2_2$$

Since iterative methods involve forward- and back-projection steps, assumptions of data acquisition, data processing, system geometries, and noise characteristic can be incorporated into the reconstruction procedure. However, iterative methods are computationally demanding, you will notice that it takes longer to get reconstruction results with iterative methods.

From mathematical point of view, projection matrix $A$ is an operator which maps from the set $x$ (*domain*) to the set $b$ (*range*):
$$A: u \to b$$
In the framework, we implemented a generic `Operator` class. The two most important methods of the `Operator` are `direct` and `adjoint` methods that describe the result of applying the operator, and its adjoint respectively, onto a compatible `DataContainer` (`AcquisitionData` or `ImageData`) input. The output is another `DataContainer` object or subclass hereof. An important special case of the `Operator` class, is the projection operator $A$ for CT, where `direct` and `adjoint` method correspond to forward- and back-projection respectively.

In [ ]:
# imports
from ccpi.astra.operators import AstraProjectorSimple 

# define the projection operator A
operator = AstraProjectorSimple(ig, ag, dev)

# forward projection
forward_projection = operator.direct(data)

# back_projection
back_projection = operator.adjoint(forward_projection)

plotter2D([forward_projection, back_projection],
          ["Forward projection", "Back projection"],
          fix_range=False,
          cmap=cmap)

In [ ]:
print("Range: {} \n".format(operator.range_geometry()))
print("Domain: {} \n".format(operator.domain_geometry()))

The `Operator` class also has a `norm` method.

In [ ]:
print("Operator norm: {}\n".format(operator.norm()))

The Framework provides a number of generic optimisation algorithms implementations. All algorithms share the same interface and behaviour. Algorithms are iterable Python object which can be run in a for loop, can be stopped and warm restarted.

The Conjugate Gradient Least Squares (CGLS) algorithm is commonly used for solving large systems of linear equations, due to its fast convergence. CGLS takes `operator`, measured data and initial value as an input.

In [ ]:
# imports
from ccpi.optimisation.algorithms import CGLS

# initial estimate - zero array in this case 
x_init = ig.allocate(0)

# setup CGLS
cgls = CGLS(x_init=x_init, 
            operator=operator, 
            data=ad)
cgls.max_iteration = 10
cgls.update_objective_interval = 1

In [ ]:
# run N interations
cgls.run(10, verbose=True)

In [ ]:
# get and visusualise the results
recon_cgls_ideal = cgls.get_output()

plotter2D([recon_cgls_ideal, recon_cgls_ideal - data],
          ["CGLS reconstruction", "Difference from ground truth"],
          fix_range=True,
          cmap=cmap)

Alternatively, tolerance can be used as a stopping criterion.

In [ ]:
# setup CGLS
cgls = CGLS(x_init=x_init, 
            operator=operator, 
            data=ad,
            tolerance=1e-4) # default 1e-6
cgls.max_iteration = 200
cgls.update_objective_interval = 10

In [ ]:
# run N interations
cgls.run(200, verbose=True)

In [ ]:
# get and visusualise the results
recon_cgls_dummy = cgls.get_output()

plotter2D([recon_cgls_dummy, recon_cgls_dummy - data],
          ["CGLS reconstruction", "Difference from ground truth"],
          fix_range=True,
          cmap=cmap)

## Adding some complexity

In the example above we worked with ideal (i.e. noise- or artifacts-free) sinogram acquired over the sufficient number of rotation positions which is not always the case with datasets obtained in real experiments. Let us take a look how both FBP and CGLS algorithms will perform on noisy and/or insufficient data.

### Noisy data 

As X-ray photons travel from an X-ray source to detector elements they interact with matter along their trajectories. In these interactions, photons are either absorbed or scattered, resulting in the attenuation of the incident X-ray. A quantitative description of the interaction of X-rays with matter is given by the Beer-Lambert law (or Beer’s law).
$$I^{l} = I^0 \mathrm{exp}\left( -\int_{l} f(g) \mathrm{d}l \right)$$
where $f(g)$ is the X-ray linear attenuation coefficient of the object at the position $g$ along a given linear X-ray trajectory $l$ from the source to the detector element. If $l$ is the entire trajectory from the source to the detector element, then $I^0$ corresponds to the X-ray intensity upon emission from the source and $I^{l}$ corresponds to the X-ray intensity upon incidence on the detector element. $I^{l}$ is typically called a transmission measurement, whereas a projection measurement is given by
$$G^{l} = -\log \left( \frac{I^{l}}{I^0} \right) = \int_{l} f(g) \mathrm{d}l$$

Ideally, $I^0$ is a single value, but real detector pixels do respond equally to photon flux. Secondly, pixels might have residual charge (so called dark current). Therefore, to convert $I^{l}$ to $G^{l}$, one needs to perform flat field correction. If $I^F$ is a flat field image (acquired with source on, without an object in the field of view) and $I^d$ is a dark field image (acquired with source off), then flat field correction is given by:
$$\frac{I-I^D}{I^F-I^D}$$

Both $I$ and $I^F$ are subject to Poisson noise due to photon counting process. Finally, detectors have limited bit depth (typically between 8 and 16 bits) which introduce additional discretization error due to the limited number of shades which can be encoded with given bit depth.

For the following example, we sinulated noisy sinogram, dark and field images for 8 bit detector (0-255).

In [ ]:
# load noisy projections, flat- and dark-field images

# filemane
pathname = data_dir
filename = 'proj.nxs'
path = os.path.join(pathname, filename)

# load projections
reader = NEXUSDataReader()
reader.set_up(nexus_file=path)
proj = reader.load_data()
# get AcquisitionGeometry
ag_noisy = reader.get_geometry()

# filemane
pathname = data_dir
filename = 'flat.nxs'
path = os.path.join(pathname, filename)

# load flat field image
reader = NEXUSDataReader()
reader.set_up(nexus_file=path)
flat = reader.load_data()

# filemane
pathname = data_dir
filename = 'dark.nxs'
path = os.path.join(pathname, filename)

# load dark field image
reader = NEXUSDataReader()
reader.set_up(nexus_file=path)
dark = reader.load_data()

In [ ]:
print(ag_noisy)

In [ ]:
plotter2D([proj, flat, dark], 
          ["Projection", "Flat field", "dark field"], 
          fix_range=True, 
          stretch_y=False,
          cmap=cmap)

All three loaded images are belong to the `AcquisitionData` class. We defined algebraic operations between objects of `AcquisitionData` class. As a result, we can perform flat/ dark-field correction as follows:

In [ ]:
# perform flat field correction and take negative logarithm
ad_noisy = -1*(((proj - dark) / (flat - dark)).log())

plotter2D([ad, ad_noisy], 
          ["Noise-free sinogram", "Noisy sinogram"], 
          fix_range=True, 
          stretch_y=False,
          cmap=cmap)

In [ ]:
# FBP reconstruction of noisy data

# create ImageGeometry 
ig_noisy = ImageGeometry(voxel_num_x=ag_noisy.pixel_num_h,
                         voxel_size_x=ag_noisy.pixel_size_h,
                         voxel_num_y=ag_noisy.pixel_num_h,
                         voxel_size_y=ag_noisy.pixel_size_h)

# configure FBP
fbp = FBP(volume_geometry=ig_noisy, 
          sinogram_geometry=ag_noisy,
          device=dev)
fbp.set_input(ad_noisy)
recon_fbp_noisy = fbp.process()

In [ ]:
plotter2D(data, 
          "Ground truth",
          cmap=cmap)

plotter2D([recon_fbp_noisy, recon_fbp_noisy-data], 
          ["Reconstruction of noisy sinogram", "Difference from ground truth"], 
          fix_range=False, 
          stretch_y=False,
          cmap=cmap)

The reconstruction above doesn't look particularly good. Let us try to reconstruct the same noisy dataset using the CGLS method. In CGLS without explicit regularisation, the number of iterations plays the role of a regularisation parameter. However, it is often unclear how many iterations is required to get 'good' reconstruction. To control how reconstruction result changes with every iteration, we will visualise intemediate reconstruction results.

In [ ]:
# projection operator
operator = AstraProjectorSimple(ig_noisy, ag_noisy, dev)

# initial estimate - zero array in this case 
x_init = ig.allocate(0)

max_iter = 20
step = 2

# setup CGLS
cgls = CGLS(x_init=x_init, 
            operator=operator, 
            data=ad_noisy)
cgls.max_iteration = max_iter

for i in range(0, max_iter // step):
    cgls.run(step, verbose=True)
    
    # get and visusualise the results
    recon_cgls_dummy = cgls.get_output()

    plotter2D([recon_cgls_dummy, recon_cgls_dummy - data],
              ["Iteration {}, objective {}".format(i * step, cgls.loss[-1]), "Difference from ground truth"],
              fix_range=True,
              cmap=cmap)

You can see that after iteration 4, reconstruction gets more noisy even though objective value keeps decreasing. After iteration 8, you cannot see significant changes in the reconstruction result.

In [ ]:
# re-run CGLS reconstruction with 4 iterations
# setup CGLS
cgls = CGLS(x_init=x_init, 
            operator=operator, 
            data=ad_noisy)
cgls.max_iteration = 4

cgls.run(4, verbose=True)
    
# get the results
recon_cgls_noisy = cgls.get_output()

### Missing angle data
Quite often acquired CT data has insufficient number of projections due to some limitations (for instance, for dose reduction in medical imaging and in-vivo synchrotron tomography of small organisms, for time-resolved imaging of fast processes etc). We will take both noise-free and noisy acquisition and choose a relatively small subset of projections, i.e. rows, using slicing. 

We will use the following mask to simulate missing-angles data acquisition:

In [ ]:
n_angles = 360
a = np.int32(np.round((np.arange(1,28) * np.arange(0,27)) / 4))
mask = np.concatenate([a, n_angles - 1 - a[::-1]])
print("Mask: {}\n".format(mask))
print("Number of angles: {}\n".format(len(mask)))

In [ ]:
# visualise missing angles acquisition
sino_masked = np.zeros_like(ad.as_array())
sino_masked[mask, :] = ad.as_array()[mask, :]

plotter2D([ad, sino_masked],
          ["All angles", "Missing angles acquisition"],
          cmap=cmap)

Now we will show some tricks how to manipulate objects in the Framework. First, we create new `AcquisitionGeometry` and `AcquisitionData` from existing.

In [ ]:
# create a copy of existing AcquisitionGeometry
ag_low = ag.clone()

Note, that `ag_low = ag` will not create a copy but will create a pointer to the `ag` object, therefore we have to use the `clone` method.

Acquisition angles in `AcquisiionGeometry` are stored as a numpy array. Below we use numpy slicing to assign new angles:

In [ ]:
# and pass new acquisition angles using slicing of numpy array.
ag_low.angles = ag.angles[mask]

print(ag_low)

Secondly, create new `AcquisitionData` from existing.

In [ ]:
# allocate new AcquisitionData from ag_low
ad_low = ag_low.allocate()

print(ad_low)

In [ ]:
# unwrap ad and use numpy slicing to create a new sinogram with low number of projections
ad_low.fill(ad.as_array()[mask,:])

plotter2D([ad, ad_low], 
          ["Full sinogram", "Missing angles sinogram"], 
          fix_range = True, 
          stretch_y = False,
          cmap=cmap)

Repeat the same manipulations for noisy data (we do not need to create a new `AcquisitionGeometry` for noisy data, because both noise-free and noisy sinograms will share the same geometry).

In [ ]:
# allocate new AcquisitionData from ag_low
ad_low_noisy = ag_low.allocate()

# unwrap ad and use numpy slicing to create a new sinogram with low number of projections
ad_low_noisy.fill(ad_noisy.as_array()[mask,:])

plotter2D([ad_low, ad_low_noisy], 
          ["Missing angles sinogram, noise-free", "Missing angles sinogram, noisy"], 
          fix_range=True, 
          stretch_y=False,
          cmap=cmap)

**Exercise 1:**

Set-up [FBP reconstruction](#fbp) of both `ad_low` and `ad_low_noisy` datasets. Compare with reconstrctions of both (noise-free and noisy) datasets with sufficient number of projections.
Hint: you do not need to create new `ImageGeometry`, you can re-use the same `ig` because we reconstruct all the datastes onto the same voxel grid.

Below you can see commented skeleton of the code you need to modify.

In [ ]:
# reconstruct noise-free data
# configure FBP
# fbp = FBP(volume_geometry=ig, 
#           sinogram_geometry=...,
#           device=dev)
# pass actual AcquisitionData
# fbp.set_input(...)
# run FBP and get results
# recon_fbp_low = fbp.process()

In [ ]:
# reconstruct noisy data
# configure FBP
# fbp = FBP(volume_geometry=ig, 
#           sinogram_geometry=...,
#           device=dev)
# pass actual AcquisitionData
# fbp.set_input(...)
# run FBP and get results
# recon_fbp_low_noisy = fbp.process()

In [ ]:
# visualise reconstruction results

# plotter2D(data, 
#           "Ground truth",
#           cmap=cmap)

# plotter2D([recon_fbp_ideal, recon_fbp_ideal-data], 
#           ["Full acquisition, noise-free", "Difference from ground truth"], 
#           fix_range=True, 
#           stretch_y=False,
#           cmap=cmap)

# plotter2D([recon_fbp_noisy, recon_fbp_noisy-data], 
#           ["Full acquisition, noisy", "Difference from ground truth"], 
#           fix_range=True, 
#           stretch_y=False,
#           cmap=cmap)

# plotter2D([recon_fbp_low, recon_fbp_low-data], 
#           ["Missing angles, noise-free", "Difference from ground truth"], 
#           fix_range=True, 
#           stretch_y=False,
#           cmap=cmap)

# plotter2D([recon_fbp_low_noisy, recon_fbp_low_noisy-data], 
#           ["Missing angles, noisy", "Difference from ground truth"], 
#           fix_range=True, 
#           stretch_y=False,
#           cmap=cmap)

**Exercise 2:**

Set-up [CGLS reconstruction](#cgls) of both `ad_low` and `ad_low_noisy` datasets. Compare with reconstrctions of both (noise-free and noisy) datasets with sufficient number of projections.

Hint: you have to create a new `operator` because `AcquisitionGeometry` have changed. You do not need to create new `ImageGeometry`.

Below you can see commented skeleton of the code you need to modify.

In [ ]:
# for noise-free dataset we will use max_iter as stopping criterion

# projection operator
# operator_low = AstraProjectorSimple(ig, ..., dev)

# initial estimate - zero array in this case 
# x_init = ig.allocate(0)

# max_iter = ...

# setup CGLS
# cgls = CGLS(x_init = ..., 
#             operator = ..., 
#             data = ...)
# cgls.max_iteration = ...
# cgls.update_objective_interval = ...

# cgls.run(..., verbose = True)
    
# get and visusualise the results
# recon_cgls_low = ...

In [ ]:
# for noisy dataset we will first try to find the best number of iterations
# max_iter = ...
# step = ...

# setup CGLS
# cgls = CGLS(x_init=..., 
#             operator=..., 
#             data=...)
# cgls.max_iteration = ...

# for i in range(0, max_iter // step):
#     cgls.run(step, verbose=True)
#     
#     # get and visusualise the results
#     recon_cgls_dummy = ...

#     plotter2D([recon_cgls_dummy, recon_cgls_dummy - data],
#               ["Iteration {}, objective {}".format(i * step, cgls.loss[-1]), "Difference from ground truth"],
#               fix_range=True,
#               cmap=cmap)

How many iterations is required to get the most interpretable results? Re-run CGLS with this number of iterations.

In [ ]:
# setup CGLS
# cgls = CGLS(x_init=..., 
#             operator=..., 
#             data=...)
# cgls.max_iteration = ...
# cgls.run(..., verbose=True)

# get and visusualise the results
# recon_cgls_low_noisy = ...

In [ ]:
# visualise reconstruction results

# plotter2D(data, 
#           "Ground truth",
#           cmap=cmap)

# plotter2D([recon_cgls_ideal, recon_cgls_ideal-data, recon_cgls_noisy, recon_cgls_noisy-data], 
#           ["Full acquisition, noise-free", "Difference from ground truth", "Full acquisition, noisy", "Difference from ground truth"], 
#           fix_range=False, 
#           stretch_y=False,
#           cmap=cmap)

# plotter2D([recon_cgls_low, recon_cgls_low-data, recon_cgls_low_noisy, recon_cgls_low_noisy-data], 
#           ["Missing angles, noise-free", "Difference from ground truth", "Missing angles, noisy", "Difference from ground truth"], 
#           fix_range=False, 
#           stretch_y=False,
#           cmap=cmap)

### Summary

In this notebook you have learn how to:
- create `AcquisitionGeometry` and `ImageGeometry`
- manipulate `AcquisitionData` and `ImageData`
- implement basic CT reconstructions, including analytic FBP and iterative CGLS reconstruction algorithms
- evaluate intermediate and final reconstruction results